# Introduction

<b>SimGBS</b> is a versatile method of <b>simulating Genotyping-By-Sequencing data (GBS)</b>. It can be implemented with any genome of choice. Users can modify different parameters to customise GBS setting, such as the choice of restriction enzyme and sequencing depth. By taking the gene-drop approach, users can also specify the demographic history and define population structure (by supplying a pedigree file). Like real sequencers, SimGBS will output data into FASTQ format.


This notebook is a [general jupyter notebook template](https://colab.research.google.com/github/ageron/julia_notebooks/blob/master/Julia_Colab_Notebook_Template.ipynb) modified to facilitate implementation of [SimGBS](https://github.com/kanji709/SimGBS.jl). For bigger analysis, we strongly recommend running SimGBS in julia directly.



# Setting up julia environment

## <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" />

1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook, then upload it to [Colab](https://colab.research.google.com/).

3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.5.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.5.3 on the current Colab Runtime...
2021-06-15 08:05:56 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.5/julia-1.5.3-linux-x86_64.tar.gz [105260711/105260711] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.5

Success! Please reload this page and jump to the next section.


# Checking the julia Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.5.3
Commit 788b2c77c1 (2020-11-09 13:37 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, znver2)
Environment:
  JULIA_NUM_THREADS = 2


In [ ]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 
  memory estimate:  32.00 MiB
  allocs estimate:  2
  --------------
  minimum time:     404.867 ms (0.00% GC)
  median time:      419.541 ms (0.11% GC)
  mean time:        431.297 ms (4.29% GC)
  maximum time:     513.695 ms (20.34% GC)
  --------------
  samples:          12
  evals/sample:     1

In [ ]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

# Install SimGBS

In [2]:
 import Pkg;Pkg.add("SimGBS")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [ ]:
cd("/content")

# Import SimGBS

In [3]:
using SimGBS

┌ Info: Precompiling SimGBS [92ef5d18-3a1f-4c68-9409-006e7edfe355]
└ @ Base loading.jl:1278


# Download example data

In [4]:
download("https://github.com/kanji709/SimGBS.jl/raw/master/example/ref.fa.gz","ref.fa.gz")
download("https://github.com/kanji709/SimGBS.jl/raw/master/example/GBS_Barcodes.txt","GBS_Barcodes.txt")
readdir()

4-element Array{String,1}:
 ".config"
 "GBS_Barcodes.txt"
 "ref.fa.gz"
 "sample_data"

# Run SimGBS example

The code below can be found in [example.jl](https://github.com/kanji709/SimGBS.jl/tree/master/example)

In [5]:
## parameters
### Step One: Generate GBS Fragments
genofile = "ref.fa.gz"
re = [SimGBS.ApeKI]; # specify the restriction enzyme to be used for virtual digestion
useChr = [1]; # specify either the number of chromosome(s) or a set of chromosome(s) to be used for simulating GBS data
useChrLen = Array{Float64}(undef, 0); # specify the length of chromsome(s) in cM to be simulated, otherwise using the entire chromosome
lower = 65; # lower bound of fragment size selection
upper = 195; # upper bound of fragment size selection
winSize = 1000000 # window size to be used to sample SNP density

### Step Two: Define Population Structure
numFounders = 100; # number of founders in the base population
endSize = 1000; # number of individuals to end up in the changingPopSize step
numGenCha = 20; # number of generations for changingPopSize function 
numGenCon = 50; # number of generations for constantPopSize function 
numGenFinal = 4; # number of final generations to be used to select individual 
numInd = 96; # number of individuals to be simulated
useWeights = Array{Float64}(undef, 0) # weights of each of contributing genetarion in the fianal population composition 
usePedigree = false;  # false if you don't use pedigree, otherwise specify the pedigree file to be used
pedFile = "newPed.ped"; # file stores the pedigree (default = "sim.ped")
pedOutput = false; # true if print out pedigree (default = false)

### Step Three: Simulate GBS Process
totalQTL = 100 # total (across all chromosomes) number of QTLs to be simulated
totalSNP = 0 # total (across all chromosomes) number of QTLs to be simulated, set [totalSNP = 0] if sampling based on density
muDensity = -2.0 # location parameter of log-Laplace distribution (for sampling SNP density)
sigmasqDensity = 0.001 # scale parameter of log-Laplace distribution (for sampling SNP density)
winSize = 1000000 # window size to be used to sample SNP density
muAlleleFreq = 0.5 # mean of sampled allele frequency
sigmasqAlleleFreq = 0.0061 # variance of sampled allele frequency
meanDepth = 20.0 # expected sequencing depth
barcodeFile = "GBS_Barcodes.txt" # file contains GBS barcodes
useChr = [1]; # specify either the number of chromosome(s) or a set of chromosome(s) to be used for simulating GBS data 

### miscellaneous
plotOutput = true; # true if plots are required
writeOutput = true; # true if outputs in text file are required
onlyOutputGBS = true; # true if only GBS data is required

## Run SimGBS
### Step One: Generate GBS Fragments
@time digestGenome(
    genofile,
    re,
    useChr,
    useChrLen,
    lower,
    upper,
    winSize,
    plotOutput,
    writeOutput,
);

### Step Two: Define Population Structure
@time definePopulation(
    numFounders,
    endSize,
    numGenCha,
    numGenCon,
    numGenFinal,
    numInd,
    useWeights,
    usePedigree,
    pedFile,
    pedOutput,
);

### Step Three: Simulate GBS Process
@time GBS(
    totalQTL,
    totalSNP,
    muDensity,
    sigmasqDensity,
    winSize,
    muAlleleFreq,
    sigmasqAlleleFreq,
    re,
    meanDepth,
    barcodeFile,
    useChr,
    plotOutput,
    writeOutput,
    onlyOutputGBS,
);


INFO: Using chromsome [1] in the simulation!
CHROMOSOME 1: 1800 GBS fragments are generated through virtual digestion using ApeKI.
CHROMOSOME 1: Estimated chromosome length equals to 0.999602 Mb.
CHROMOSOME 1: Subsetting chromosome is not required. Keeping all 1800 available GBS fragments.
INFO: 354 out of 1800 GBS fragments are selected after size-selection with lower and upper thresholds equals to [65, 195].
INFO: Expected sequencing coverage based on 354 selected GBS fragments is approximately 4.56%.


┌ Info: Precompiling GR_jll [d2c73de3-f751-5644-a686-071e5b155ba9]
└ @ Base loading.jl:1278


133.091351 seconds (55.75 M allocations: 2.805 GiB, 0.93% gc time)
CHANING POP SIZE GEN 10: Done!
CHANING POP SIZE GEN 20: Done!
CONSTANT POP SIZE GEN 10: Done
CONSTANT POP SIZE GEN 20: Done
CONSTANT POP SIZE GEN 30: Done
CONSTANT POP SIZE GEN 40: Done
INFO: Collecting 28 individual at Gen 47
INFO: Collecting 16 individual at Gen 48
INFO: Collecting 27 individual at Gen 49
INFO: Collecting 25 individual at Gen 50
  4.688148 seconds (10.76 M allocations: 522.343 MiB, 13.10% gc time)
INFO: A total of 100 QTLs are sampled randomly across 1 chromosome(s).
CHROMOSOME 1: 201655 SNPs sampled with average SNP density = 0.135 (window size = 1000000 bp).
CHROMOSOME 1: Found 8988 SNPs on 354 GBS fragments, with an average of 25.39 SNPs per GBS fragment.
[INFO]: a total of 201655 SNPs were sampled.
[INFO]: 8988 SNPs captured by selected GBS SNPs.
CHROMOSOME 1: Filling haplotypes!
CHROMOSOME 1: DONE!
  2.311320 seconds (16.46 M allocations: 696.797 MiB, 7.63% gc time)
[INFO] On Lane 1 of Flowcell A

# Outputs

<b>GBS Fragments</b>

- RawFrag.txt: raw GBS fragments following in slico digestion
- GBSFrag.txt: selected GBS fragments after fragment size-selection
- GBSCoverage.txt: genomic coverage of GBS fragments
- snpFragGBS.txt: GBS fragments that contains SNPs

<b>Variants</b>

- qtlGeno.txt: QTL genotype (number of individual x number of QTL loci)
- snpGeno.txt: SNP genotypes (number of individual x number of QTL loci)
- qtlInfo.txt: information about QTL, including chromosome, position and allele frequency
- snpInfo.txt: information about SNPs, including chromosome, position and allele frequency
- shortHap.txt: short-haplotype created by GBS fragments (i.e., SNPs captured within each GBS fragment)
- readDepth.txt: number of copies per GBS fragment

<b>GBS Data</b>

- keyFileABC12AAXX1.txt: pseudo-information about GBS sample, including flowcell, lane, barcode, sample name, plate, row and column
- ABC12AAXX1fastq.txt.gz: simulated GBS sequences

In [7]:
readdir() # see outputs

23-element Array{String,1}:
 ".config"
 "ABC12AAXX_1_fastq.txt.gz"
 "GBSCoverage.png"
 "GBSCoverage.txt"
 "GBSFrag.txt"
 "GBSFragSize.png"
 "GBS_Barcodes.txt"
 "RawFrag.txt"
 "RawFragSize.png"
 "keyFile_ABC12AAXX_1.txt"
 "qtlAF.png"
 "qtlGeno.txt"
 "qtlInfo.txt"
 "readDepth.txt"
 "ref.fa.gz"
 "sample_data"
 "shortHap.txt"
 "snpAF.png"
 "snpDepth.txt"
 "snpFragGBS.txt"
 "snpGeno.txt"
 "snpInfo.txt"
 "snpPerTag.png"

# Available functions

## Define genome

In [ ]:
digestGenome("ref.fa.gz", [SimGBS.ApeKI], [1], Array{Float64}(undef,0), 65 ,195, 1000000, false, true)

INFO: Using chromsome [1] in the simulation!
CHROMOSOME 1: 1800 GBS fragments are generated through virtual digestion using ApeKI.
CHROMOSOME 1: Estimated chromosome length equals to 0.999602 Mb.
CHROMOSOME 1: Subsetting chromosome is not required. Keeping all 1800 available GBS fragments.
INFO: 354 out of 1800 GBS fragments are selected after size-selection with lower and upper thresholds equals to [65, 195].
INFO: Expected sequencing coverage based on 354 selected GBS fragments is approximately 4.56%.


## Define population

In [ ]:
 definePopulation(100, 500, 20, 100, 4, 96,  Array{Float64}(undef,0), false, "sim.ped", false);

CHANING POP SIZE GEN 10: Done!
CHANING POP SIZE GEN 20: Done!
CONSTANT POP SIZE GEN 10: Done
CONSTANT POP SIZE GEN 20: Done
CONSTANT POP SIZE GEN 30: Done
CONSTANT POP SIZE GEN 40: Done
CONSTANT POP SIZE GEN 50: Done
CONSTANT POP SIZE GEN 60: Done
CONSTANT POP SIZE GEN 70: Done
CONSTANT POP SIZE GEN 80: Done
CONSTANT POP SIZE GEN 90: Done
INFO: Collecting 20 individual at Gen 97
INFO: Collecting 29 individual at Gen 98
INFO: Collecting 29 individual at Gen 99
INFO: Collecting 18 individual at Gen 100


## GBS

In [6]:
GBS(100, 0, -2.0, 0.001, 1000000, 0.5, 0.001, [SimGBS.ApeKI], 20.0, "GBS_Barcodes.txt", [1], false, true, true)

INFO: A total of 100 QTLs are sampled randomly across 1 chromosome(s).
CHROMOSOME 1: 203470 SNPs sampled with average SNP density = 0.135 (window size = 1000000 bp).
CHROMOSOME 1: Found 9131 SNPs on 354 GBS fragments, with an average of 25.79 SNPs per GBS fragment.
[INFO]: a total of 203470 SNPs were sampled.
[INFO]: 9131 SNPs captured by selected GBS SNPs.
CHROMOSOME 1: Filling haplotypes!
CHROMOSOME 1: DONE!
  1.741354 seconds (15.84 M allocations: 662.936 MiB, 19.60% gc time)
[INFO] On Lane 1 of Flowcell ABC12AAXX: Average GBS read depth equals to 19.07.
[INFO] On Lane 1 of Flowcell ABC12AAXX: Generating GBS data for 96 samples.
INFO: A total of 648196 GBS reads genertaed.


# Overview

For more information, please visit the [documentation](https://kanji709.github.io/SimGBS.jl/dev/) page.



# Citing

Please cite the following if you use SimGBS.jl,

- Hess, A. S., M. K. Hess, K. G. Dodds, J. C. Mcewan, S. M. Clarke, and S. J. Rowe. "A method to simulate low-depth genotyping-by-sequencing data for testing genomic analyses." Proc 11th World Congr Genet Appl to Livest Prod 385 (2018).




# What is next?

The following tools are recommended for downstream Analyses of GBS data,

[snpGBS](https://github.com/AgResearch/snpGBS): a simple bioinformatics workflow to identify single nucleotide polymorphism (SNP) from Genotyping-by-Sequencing (GBS) data.

[KGD](https://github.com/AgResearch/KGD): R code for the analysis of genotyping-by-sequencing (GBS) data, primarily to construct a genomic relationship matrix for the genotyped individuals.

[GUSLD](https://github.com/AgResearch/GUS-LD): An R package for estimating linkage disequilibrium using low and/or high coverage sequencing data without requiring filtering with respect to read depth.

[SMAP](https://gitlab.com/truttink/smap) a software package that analyzes read mapping distributions and performs haplotype calling to create multi-allelic molecular markers.